In [1]:
from Bio import SeqIO
import pandas as pd

df = pd.DataFrame()

for seq_record in SeqIO.parse("sequence.fasta", "fasta"):
    print(repr(seq_record.seq))
    print("Length of sequence is", len(seq_record))
    seqSeries = pd.Series(list(str(seq_record.seq)))


Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN', SingleLetterAlphabet())
Length of sequence is 120883175


In [2]:

searchedPattern = "CTC" 
searchedSeries = pd.Series([x for x in searchedPattern])
print("Length of pattern is", len(searchedSeries))

Length of pattern is 3


In [3]:
seqSeries = seqSeries[10000000:10001000]
seqSeries

10000000    C
10000001    T
10000002    C
10000003    T
10000004    T
           ..
10000995    C
10000996    T
10000997    T
10000998    C
10000999    T
Length: 1000, dtype: object

In [14]:
def nativePatternSearch(sequence, pattern):
    #Simple linear Pattern Searching
    indexList = list() # list for storing matched pattern starting index 
    patternLen = len(pattern)
    for keyParent, valueParent in sequence.items(): # iter over every char in sequence (split key/value)
        for keyChild, valueChild in pattern.items(): # iter over every char in searched pattern (split key/value)
            try:
                # check if following individual chars matches all chars in searched pattern break internal for loop if a single char does not match. 
                # if current index char and following char fulfill searched pattern append to indexlist
                if sequence[keyParent + keyChild] != valueChild: break 
                elif (patternLen == (keyChild + 1)): indexList.append(keyParent) 
            except: break
    return indexList

print(nativePatternSearch(seqSeries, searchedSeries))

[10000000, 10000021, 10000253, 10000317, 10000378, 10000453, 10000473, 10000549, 10000561, 10000648, 10000685, 10000761, 10000817, 10000819, 10000825, 10000889, 10000929]


In [15]:
def badBoyerMoorePatternSearch(sequence,pattern):
    patternLen = len(pattern)
    sequenceLen = len(sequence)

    preprocessingDic = dict()
    for key, value in pattern.items():
        preprocessingDic[value] = key 
    for key, value in preprocessingDic.items():
        if(patternLen - value - 1 != 0): preprocessingDic[key] = patternLen - value - 1

    indexlist = list()    
    keyParent = sequence.index[0]+ (patternLen - 1)  # set KeyParent as the index value of first backward searchable index 
    
    while keyParent < sequence.index[0] + sequenceLen  :  # loop all char elements from frist backward searchable index
        valueParent = sequence[keyParent] #set valueParent value of the squence element in question
        
        keyChild = patternLen - 1 # set KeyChild as the last index
        while keyChild >= 0 : #check backward 
            backward = patternLen - 1 - keyChild 
            # if sequence element in question is not same as element as in pattern 
            # shift forward to align common char in pattern (number of shift in preprocessDic)
            # break useless checking
            if sequence[keyParent - backward] != pattern[keyChild] : 
                keyParent += preprocessingDic.get(valueParent,patternLen)
                break
            keyChild -= 1 # decrement if char matches
        if keyChild < 0: 
            indexlist.append(keyParent -(patternLen - 1)) #append index of first char of matched pattern
            keyParent += 1
    return indexlist

print(badBoyerMoorePatternSearch(seqSeries, searchedSeries))

[10000000, 10000021, 10000253, 10000317, 10000378, 10000453, 10000473, 10000549, 10000561, 10000648, 10000685, 10000761, 10000817, 10000819, 10000825, 10000889, 10000929]
